# Bayesian Learning
Welcome to the 4th session on Machine Learning practicals. In this session, we'll learn about a classic Machine Learning method - the Bayesian Learning method. The Bayesian approach towards problem solving is completely based on probabilities. This aligns with the idea behind Machine Learning, since in the latter too, every prediction is visualised as a probability. 

For example, in logistic regression, the final output (the output from the sigmoid function) lies between 0 and 1. We visualize the output as the *probability that the prediction is 1*, and `1-output` as the *probability that the prediction is 0*. And we create a threshold, meaning any prediction less than 0.5 is not acceptable as a prediction of 1. (Or in other words, you should be atleast 50% sure that the prediction is a 1). Let us take an example - suppose a logistic regression model returns 0.8 as the output for a particular input, the model is 80% sure that the prediction is equal to a 1, and 20% sure that the prediction is equal to a 1. We do qualify this prediction as a 1. However, the aim of training a model is to adjust the parameters in such a way that this probability is maximized for all inputs.

So Bayesian Learning is an important stepping stone towards Machine Learning. However, its a very crude form of Learning - or in other words, a very superficial form of learning - it learns the pattern of *distribution* of data directly, and nothing else. No intrinsic features are learned, all learned features are independent and remain uncorrelated, etc. But still, Many researchers still find Bayesian Learning a useful tool to use in ensemble with modern Machine Learning techniques. This is because despite its straight forward approach, it is the purest form of data representation - it gathers * all * the information that the distribution of the data provides - no more and no less. We will look into these details in the following sections. 

In this session, we will learn about 2 important Bayesian Learning approaaches - the **Naive Bayes Classifier** and **Markov Model**. So let us begin!

## The Naive Bayes Classifier
The Naive Bayes Classifier simply provides us with the information - given a dataset, what is the probability that a particular model gives correct classification on the data.

You must have come across the Bayes Equation:

$ P(E|D) = \frac{P(E) P(D|E)}{P(D)}$

The D refers to our data. We would always have some data that we wish to build a model upon. The $E$ refers to the *evidence*, or simply the prediction of the model. So in simpler terms - 


$ P(pred|data) = \frac{P(pred) P(data|pred)}{P(data)}$

For each of different choices of predictions, the Bayes Theorem holds:

$ P(pred=1|data) = \frac{P(pred=1) P(data|pred=1)}{P(data)}$

$ P(pred=0|data) = \frac{P(pred=0) P(data|pred=0)}{P(data)}$

... and so on.

But these math equations don't make a lot of sense. Let us try to get an intuitive understanding of their meanings. 

1. P(data) simply means - what is the probability that a particular data results in the correct prediction. Obviously, one datapoint may have many features. This can be represented as `P(data) = P(feature1 AND feature2 AND .... AND featureN)`, meaning, what is the probability of getting a correct prediction when value of feature1 is such, the value of feature2 is such, and so on. In Probability, when you want to find the probability of *a combination* of different occurences, you can simply multiply the individual probabilities. 

 $P(data = (f1,f2,f3,....fN)) = P(f1)P(f2)...P(fN)$

 P(f1) simply means - what is the probability of f1 attaining a particular value. This can be found out from the dataset itself, using a *frequentist approach*, meaning, out of all the data, how many of feature f1's have *one* particular value.


2. So, $ P(data|pred)$ means that if whenever the prediction has a value, what would be the probability that if that particular data is passed in the model, this output is achieved. We will look at how to calculate this in the following subsection. T

3. $P(pred)$ is simply the probability of occurence of a particular prediction. Basically, the fraction of data which has a particular *pred* value as the output.

4. $P(pred|data)$ means, given a data point, having a particular set of features, what is the probability of obtaining a prediction, which is what we want to find out. Infact, this is what all Machine Learning wishes to find out. Even in logistic regression, we build a model that can predict an output with high probability, given some inputs. 

The proof of the Bayes theorem is actually quite simple, and can be found on the internet, or in the lecture classes as well!

But let us see how to build a Naive Bayes Classifier. So first of all we would need some data. Let us try to pick an interesting classification problem. 

Let us pick the [Airplane Passenger Satisfaction](https://www.kaggle.com/teejmahal20/airline-passenger-satisfaction), which contains data about how satisfied passengers have been by the service of their respective flights. They have provided information about the flights and the passengers themselves, such as the class in which they traveled, their experience of food, in-flight entertainment, etc. This is a binary classification problem, meaning the experience reported is either satisfactory, or dissatisfactory. This dataset can be found on kaggle.com, and so, like always, we would need to upload the *kaggle.json* file. You must have it downloaded on your system from the last sessions. If not, you can download a fresh  Run the cell below to upload the kaggle.json file. 

In [ ]:
%cd 
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json 

!kaggle datasets download -d teejmahal20/airline-passenger-satisfaction
!unzip airline-passenger-satisfaction.zip 
!mkdir airlines && mv test.csv airlines && mv train.csv airlines
%cd airlines
!ls

/root


Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 72 bytes
airline-passenger-satisfaction.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  airline-passenger-satisfaction.zip
  inflating: test.csv                
  inflating: train.csv               
mkdir: cannot create directory ‘airlines’: File exists
/root/airlines
test.csv  train.csv


So if you see, this dataset contains two csv files, namely the training dataset and the testing dataset. Let us import the pandas library to read these files, and visualize the dataset.

For the sake of simplicity, we've removed all continuous variables, and only retained categorical variables (i.e. the variables that only take values among finite number of categories), otherwise the concept of probability of occurence of an event doesn not make sense. Ideally, a continuous variable can take infinite values, so how do you find the chances of occurence of a particular value?!

These variables are - 'Departure Delay in Minutes','Arrival Delay in Minutes',and 'Flight Distance', 'Age'

In [ ]:
import pandas as pd

In [ ]:
train=pd.read_csv('train.csv',index_col=0).drop(['id','Departure Delay in Minutes','Arrival Delay in Minutes','Flight Distance','Age'],axis=1) 
test=pd.read_csv('test.csv',index_col=0).drop(['id','Departure Delay in Minutes','Arrival Delay in Minutes','Flight Distance','Age'],axis=1)
len(train),len(test)

(103904, 25976)

In [ ]:
train.head()

,Gender,Customer Type,Type of Travel,Class,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,satisfaction
0,Male,Loyal Customer,Personal Travel,Eco Plus,3,4,3,1,5,3,5,5,4,3,4,4,5,5,neutral or dissatisfied
1,Male,disloyal Customer,Business travel,Business,3,2,3,3,1,3,1,1,1,5,3,1,4,1,neutral or dissatisfied
2,Female,Loyal Customer,Business travel,Business,2,2,2,2,5,5,5,5,4,3,4,4,4,5,satisfied
3,Female,Loyal Customer,Business travel,Business,2,5,5,5,2,2,2,2,2,5,3,1,4,2,neutral or dissatisfied
4,Male,Loyal Customer,Business travel,Business,3,3,3,3,4,5,5,3,3,4,4,3,3,3,satisfied


As you can see, each of the features (all columns except the target variable - 'satisfaction') have either one of finite number of values. For example, 'Gender' only takes one of two values - Male or Female. 

But, notice, many of these features have strings as their categories. We know, that computers cannot understand strings. So we need to convert them into numbers. We define a function that converts all values into numerical categories, ranging from 0 to num_categories-1 (num_categories categories in total). Lets call this funtion *categorify*!

In [ ]:
def categorify(df):
    df.dictionary={}
    for col in df: 
        numerical_categories = {k:i for i,k in enumerate(sorted(df[col].unique()))}
        df[col]=[numerical_categories[i] for i in df[col]]
        df.dictionary[col]=numerical_categories

categorify(train)
categorify(test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


So now, if you take a look at your dataframes, you'll see that all categories are now 0,1,2,...and so on. Even the target (satisfaction) column!

In [ ]:
train.head()

,Gender,Customer Type,Type of Travel,Class,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,satisfaction
0,1,0,1,2,3,4,3,1,5,3,5,5,4,3,3,4,5,5,0
1,1,1,0,0,3,2,3,3,1,3,1,1,1,5,2,1,4,1,0
2,0,0,0,0,2,2,2,2,5,5,5,5,4,3,3,4,4,5,1
3,0,0,0,0,2,5,5,5,2,2,2,2,2,5,2,1,4,2,0
4,1,0,0,0,3,3,3,3,4,5,5,3,3,4,3,3,3,3,1


For convenience, we've also added a method in the dataframe, called *dictionary*, which will help us identify which category refers to which original category!

In [ ]:
train.dictionary

{'Baggage handling': {1: 0, 2: 1, 3: 2, 4: 3, 5: 4},
 'Checkin service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Class': {'Business': 0, 'Eco': 1, 'Eco Plus': 2},
 'Cleanliness': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Customer Type': {'Loyal Customer': 0, 'disloyal Customer': 1},
 'Departure/Arrival time convenient': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Ease of Online booking': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Food and drink': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Gate location': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Gender': {'Female': 0, 'Male': 1},
 'Inflight entertainment': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Inflight service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Inflight wifi service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Leg room service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'On-board service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Online boarding': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Seat comfort': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Type o

This is a dictionary, a python object which indexes all values in the form of `{key: value}`. If you type in `dictionary[key]`, you will get the value. So its a great method to store and retreive information. Its just like a real word dictionary. You look up information through a key, which is the word itself. So you want to know the meaning of a word, you don't search for the meaning itself, but the word. And besides the word, we find the definition too! 

So now that we are set up with our data, let us build the probabilities. We need to find 3 different probabilities - $P(pred)$,$ P(data)$ and $P(data|pred).$ Using these, we will find $P(pred|data)$. If you're unclear with the meanings of these terms, *Please* go back up and read it through again.

Let us start with building $P(pred)$. Meaning, what is the probability of getting a particular prediction (0 or 1, which correspond to the satisfaction of the passenger) (Go take a look at the 'satisfaction' key in the dataframe's dictionary, which we printed just above this cell). 

This probability can be simply found by counting the fraction of the number of times in the whole dataset, that the prediction was 1 or 0.

In [ ]:
def get_ppred(df,tgt='satisfaction'):
    f"get P(pred) as a dictionary, where tgt is the target variable and does not count as a feature." #this is a comment for the user
    p_pred=df[tgt].value_counts().to_dict()
    for p in p_pred: p_pred[p]/=len(df) # convert the frequencies into a ratio of occurence
    return p_pred

In [ ]:
p_pred = get_ppred(train,'satisfaction')
p_pred

{0: 0.5666673082845703, 1: 0.43333269171542965}

We see, that `p_pred` (which stands for $P(pred)$), is a dictionary, which tells us the probabilities of occurences of all possible predictions (that is, 0 or 1). 

Next, let us calculate $P(data)$. This is a tricky one. Our data is made up of multiple features, like Gender, Class, Seat Comfort, etc. As we mentioned above, we simply need to calculate the probability of occurence of all these features separately. In the end, we multiply the probabilities of all features. So for now, let us calculate the individual probabilities of all features (except ofcourse, the target class Satisfaction) 


In [ ]:
def get_pdata(df,tgt='satisfaction'): return {k:get_ppred(df,tgt=k) for k in df.keys().drop(tgt)}

In [ ]:
p_data=get_pdata(train)
p_data

{'Baggage handling': {0: 0.06965083153680321,
  1: 0.1108811980289498,
  2: 0.19856790883892825,
  3: 0.3597840314136126,
  4: 0.2611160301817062},
 'Checkin service': {0: 9.624268555589775e-06,
  1: 0.1240568216815522,
  2: 0.12408569448721897,
  3: 0.27377194333230676,
  4: 0.2796331228826609,
  5: 0.19844279334770556},
 'Class': {0: 0.4779892978133662,
  1: 0.449886433631044,
  2: 0.07212426855558977},
 'Cleanliness': {0: 0.0001154912226670773,
  1: 0.12817600862334463,
  2: 0.15525870033877426,
  3: 0.23650677548506313,
  4: 0.2615779950723745,
  5: 0.2183650292577764},
 'Customer Type': {0: 0.8173217585463505, 1: 0.18267824145364953},
 'Departure/Arrival time convenient': {0: 0.05100862334462581,
  1: 0.14915691407453033,
  2: 0.16545080073914384,
  3: 0.1729096088697259,
  4: 0.2458615645210964,
  5: 0.21561248845087774},
 'Ease of Online booking': {0: 0.04318409300893132,
  1: 0.16866530643671082,
  2: 0.23118455497382198,
  3: 0.2353037419156144,
  4: 0.18835655990144748,
  5: 

This is again a dictionary, with individual features as the keys in the dictionary. The value of each key is again a probability, which contains the probability of occurence of each value that the feature can take. For example, the 'Customer Type' feature takes 2 values (0 and 1) and so we have calculated the probability of occurence of each of these values. How? by simply calculating the fraction of times these two features occur in the whole training dataset

In [ ]:
p_data['Customer Type']

{0: 0.8173217585463505, 1: 0.18267824145364953}

In fact, you can ensure that the sum of all these probabilities for any feature is always 1.

In [ ]:
sum(p_data['Customer Type'].values())

1.0

Let us run a small snippet of code to ensure the sum of probabilities of all features is equal to 1. In the code snippet below, we ensure that the difference between 1 and the sum of all probabilities is almost negligible (it might not be 0, because the individual probabilities may have lower precision. For example, a 0.33333333333333333 + 0.66666666666666666 is in all practical sense, equal to 1. But according to the computer, not equal to 1, but a 0.99999999999999999 .

In [ ]:
for key in p_data: 
    assert abs(sum(p_data[key].values())-1)<1e-6, f"probabilities of the key: {key} does not sum up to 1"

Now let us define P(data|pred). This is final component of our model. This can be thought of sub-cases of P(data) itself. Basically, it means - what is the probability of the occurence of a data, given pred assumes a certain value?! So, if you look at P(data), it is the collection of the probabilities of occurence of different features for all the datapoints. 

Now if we calculate the occurences of a feature only for a particular target(pred) value, we get P(data|pred)

Obviously, if you add up the P(data|pred) for all possible prediction values, it will result in the corresponding P(data) itself. We define `get_pdata_given_pred`, which calculates this for us.

In [ ]:
def get_pdata_given_pred(df,tgt='satisfaction'): return {k:calc_pdata_given_pred(df,tgt,key=k) for k in df[tgt].unique()}

def calc_pdata_given_pred(df,tgt,key):
    ret={f:None for f in df if f!=tgt}
    l=len(df)
    df=df[df[tgt]==key]
    for f in ret:
        conditional_probs=df[f].value_counts().to_dict()
        for o in conditional_probs: conditional_probs[o]/=l
        ret[f]=conditional_probs
    return ret

In [ ]:
p_data_given_pred = get_pdata_given_pred(train)
p_data_given_pred

{0: {'Baggage handling': {0: 0.048920157068062825,
   1: 0.07812018786572221,
   2: 0.1517362180474284,
   3: 0.1867589313212196,
   4: 0.10113181398213736},
  'Checkin service': {0: 9.624268555589775e-06,
   1: 0.09437557745611333,
   2: 0.09277794887588543,
   3: 0.15104327071142593,
   4: 0.15102402217431476,
   5: 0.07743686479827533},
  'Class': {0: 0.14614451801663073,
   1: 0.3661456729288574,
   2: 0.05437711733908223},
  'Cleanliness': {0: 0.0001154912226670773,
   1: 0.10302779488758855,
   2: 0.12218971358176779,
   3: 0.13436441330458884,
   4: 0.12154488758854327,
   5: 0.08542500769941484},
  'Customer Type': {0: 0.42722128118263014, 1: 0.13944602710194026},
  'Departure/Arrival time convenient': {0: 0.026755466584539575,
   1: 0.07658030489682784,
   2: 0.09175777640899292,
   3: 0.0971377425315676,
   4: 0.1505235602094241,
   5: 0.12391245765321836},
  'Ease of Online booking': {0: 0.01450377271327379,
   1: 0.1052413766553742,
   2: 0.16091777024946105,
   3: 0.162765

`p_data_given_pred` is simply a dictionary with keys as the different possible predictions (0 and 1), each of whose values are a dictionary containing the corresponding P(data) dictionaries (dictionary containing information about the probability of occurence of each individual feature value)

Let us also verify that the for each feature, the sum of $P(data|pred)$ for each individual prediction is equal to the overall $P(data)$

In [ ]:
for feature in train.keys().drop('satisfaction'): 
    assert p_data_given_pred[1][feature].get(0,0) + p_data_given_pred[0][feature].get(0,0) == p_data[feature].get(0)

#### Lessons in Python: The `get` method.

We've seen how we can access a particular item of a list of a dictionary, using square brackets. Internally, this is possible by defining a `def __get__` (pronounced dunder get) method in the respective class definition (of the list, dictionary, etc.). In addition to `__get__`, dictionaries also have a `def get` method defined, which can take a key, and returns the associated value. Additionally, if a particular key is not found, then you can decide what default value is returned. By default, it is `None`. So the syntax is as follows:

`dict.get(key,default=None)`

Here, we've write the code to get the value of the `0` key, and if the 0 key does not exist (which is the case when the associated probability is 0), we choose to return a default value of 0 itself.

Finally, let us now define a function that can caluculate the final predictions. 

In [ ]:
def get_prediction_accuracy(df,tgt,p_data,p_pred,p_data_given_pred):
    probability_of_being_1=[]
    probability_of_being_0=[]
    features=df.keys().drop(tgt)
    for _,row in df.iterrows(): 
        probability_of_being_1.append(calculate_prob(row,features,p_data,p_pred,p_data_given_pred,tgt=1))
        probability_of_being_0.append(calculate_prob(row,features,p_data,p_pred,p_data_given_pred,tgt=0))
    
    pred=[p_one>=p_zero for p_one,p_zero in zip(probability_of_being_1,probability_of_being_0)]
    return (pred==df[tgt]).mean()

def calculate_prob(row: pd.Series,features,p_data,p_pred,p_data_given_pred,tgt=1):
    p_pred_tgt = p_pred[tgt] #p_pred for target=1
    p_data_1=1. #initializing value
    p_data_given_pred_1 =1. #initialize

    for f in features:
        p_data_1*=p_data[f][row[f]]
        p_data_given_pred_1*=p_data_given_pred[tgt][f].get(row[f],0)

    return p_pred_tgt*p_data_given_pred_1/p_data_1

Let us check the prediction accuracy on the training and testing dataset

In [ ]:
get_prediction_accuracy(train,'satisfaction',p_data,p_pred,p_data_given_pred)

0.8257141207268247

In [ ]:
get_prediction_accuracy(test,'satisfaction',p_data,p_pred,p_data_given_pred)
#note that p_data,p_pred and p_data_given_pred are probabilities corresponding to the training dataset, not the testing dataset. 
#THis is because we build our model using the training dataset only. Testing dataset is not used for finding parameters.

0.7264782876501386

So there you have it. You have built a Naive Bayes Classifier from scratch. Obviously, we don't use this classifier independently in real word applications anymore, because there exist much more sophisticated models, but it is an important concept, that somehow made way for more advanced techniques. And even today, you can find a lot of research combining modern AI techniques with traditional probabilistic techniques.


This is a good time to understand why this classifier does not perform exceptionally well. Or, to put it in better words, why modern techniques can easily outperform such a probabilistic model.

Modern models can understand relations between different features, which can be helpful to understand which features are more meaningful and important than others. Understanding relations can also help derive much complex information from the data, the lack of which causes the Bayesian model to be very superficial, since every feature is considered independently, and no analysis is done over the feature. The frequency of occurence is the sole factor that drives the predictions.





## Markov Chains
 
<figure><center>
<img src='https://drive.google.com/uc?id=12QN6EHTg_-fFxX2fYAhdu66-elyO1GNU' width='20%'>
<figcaption>Andrei A. Markov - Russian Mathematician</figcaption>

</center></figure>

Now let us look at another interesting Bayesian Model, called *Markov Model*. Its a fairly simple probabilistic approach. Markov Models, also known as Markov Chains, are used to predict continuously from a set of possible predictions, and this completely depends on probabilities. To put the whole idea in one sentence, A Markov Model tries to give the next prediction given a current prediction. So given a prediction, we try to find out what is the probability of the next prediction. So based on these probabilities, the next prediction is made. Predictions with more probability are more likely to be made.

Let us understand this with an example. Suppose you know that today is a Cloudy Day, what is the probability of a Sunny Day tomorrow, or a windy day, or a Raining day. How is this calculated? By analysing *data*. Suppose you gather data on the weather conditions of each day. The probability of the occurence of a particular weather condition is simply calculated by a probabilistic model - how many times, of all days in the data that the weather was like today, was the weather of the next day Sunny? or windy?. More occurences of Sunny after a day in the past, on which there was weather condition was similar to today,  higher the  probability of the occurence of a sunny weather tomorrow too. Hope the idea is clear. This is pretty straighforward, and makes practical sense too. 

<figure><center>
<img src='https://drive.google.com/uc?id=1dAjzO1-DLj-BYGYC8v1qmjjqViGI0s1h' width='40%'>
</center></figure>

This modeling technique can be used in many scenarios. Obviously it can be used to predict a sequence of occurences, like the weather conditions. But there are some more advanced and interesting applications. For example, You can create an **Auto text generator** using this. Let us see how this works:


### Case Study: Auto Text Generator

The idea is the same, suppose I start with a single word. We will build a Markiv Chain Model, that predicts the next word by analysing data. The data can be a huge corpus of text. Suppose the first word is "The". We find out the occurence of different words after "The" in the data. The word that occurs the highest number of times has a higher chance of being the next word. Lets say, that 60% of the times, the word that follows "The" is "Apple", 20% of the times is "Banana", 10% of times "Orange" and 10% of the times "Horse". So out of 100 times the word "The" appears in our predicted text, the Markov model would predict the next word to be "Apple" approximately 60 times, and so on.

Let us pick a random text as our data. We use the most boring text there could be ([for real, apparantly!](https://www.brade.zone/2008/09/13/boring/)). 


In [ ]:
input=f"""I am writing something. Yes, I plan to make it the most boring thing ever written. I go to the store. A car is parked. Many cars are parked or moving. Some are blue. Some are tan. They have windows. In the store, there are items for sale. These include such things as soap, detergent, magazines, and lettuce. You can enhance your life with these products. Soap can be used for bathing, be it in a bathtub or in a shower. Apply the soap to your body and rinse. Detergent is used to wash clothes. Place your dirty clothes into a washing machine and add some detergent as directed on the box. Select the appropriate settings on your washing machine and you should be ready to begin. Magazines are stapled reading material made with glossy paper, and they cover a wide variety of topics, ranging from news and politics to business and stock market information. Some magazines are concerned with more recreational topics, like sports card collecting or different kinds of hairstyles. Lettuce is a vegetable. It is usually green and leafy, and is the main ingredient of salads. You may have an appliance at home that can quickly shred lettuce for use in salads. Lettuce is also used as an optional item for hamburgers and deli sandwiches. Some people even eat lettuce by itself. I have not done this. So you can purchase many types of things at stores.

If I drive around, I sometimes notice the houses and buildings all around. There are also pieces of farm land that are very large. Houses can be built from different kinds of materials. The most common types are brick, wood, and vinyl or synthetic siding. Houses have lawns that need to be tended. Lawns need to be mowed regularly. Most people use riding lawnmowers to do this. You can also use a push mower. These come in two varieties: gas-powered and manual. You don’t see manual push-mowers very much anymore, but they are a good option if you do not want to pollute the air with smoke from a gas-powered lawnmower. I notice that many families designate the lawnmowing responsibility to a teenager in the household. Many of these teenagers are provided with an allowance for mowing the yard, as well as performing other chores, like taking out the trash, washing the dishes, making their bed, and keeping the house organized. Allowances are small amounts of money given by parents to their children, usually on a weekly basis. These usually range from 5 dollars to 15 dollars, sometimes even 20 dollars. Many parents feel that teenagers can learn financial responsibility with this system.

Now I will talk about farm land. Farm land can be identified by some common features. They almost always consist of a very large patch of dirt with small green plants lined up in very long rows. You may sometimes see farm equipment riding over these rows, like tractors or combines. These machines help farmers grow more crops in less time. They are a very helpful invention. Some different types of crops are soybeans, cotton, corn, tomatoes, tobacco, and lettuce (which I mentioned earlier). Most crops are used as food, and can be defined as either fruits or vegetables. Some are commonly eaten raw, after being rinsed in water to remove any dirt. Some are often cooked, which helps give them a more pleasant taste and makes them easier to chew. A very versatile vegetable is the potato. It can be eaten raw, or it can be cooked in a variety of ways. They can be baked, and many people like to add butter to them. They can be mashed, and a lot of times brown gravy or milk gravy is poured on top of them. They can be cut into thin strips and fried. Typically a large amount of grease is required to prepare potatoes in this style, but they are easy to make and easy to eat. You can order them at several fast-food restaurants. Potatoes can also be boiled, stewed, and scalloped. There is a wide variety of options available to you when cooking potatoes.

Some other types of crops grown on farm land are used for other purposes. Cotton is used to make clothing (which I also mentioned earlier). It is a very versatile and inexpensive material for clothes. Such items as shirts, pants, socks, and underwear can be made from cotton. The process of converting cotton from a cotton plant to clothing is fairly complicated. Today, cotton is harvested more efficiently through the use of the cotton gin, invented by Eli Whitney many years ago.

Tobacco is another type of crop. It is used in making cigarettes. A lot of people smoke cigarettes, even though many medical sources have identified them as harmful to people’s health. Warnings are printed on cigarette packages reminding people of possible dangers resulting from smoking. Cigarettes are available in several brands, including Marlboro, Salem, and Virginia Slims. There is a brand called Kool, but I don’t know whether they are still available at most outlets. Tobacco farming is a large industry, and currently there is debate about it. Recently the government decided on some regulations that cost tobacco companies a large amount of money.

If you notice, some farm lands have animals living on them. Most of these are cows, and there are also pigs, sheep, and goats living on farms. Some are raised for the milk they provide. This milk goes through several processes to ensure that it is not contaminated before it is made available to consumers at stores (which I mentioned earlier). Another use for farm animals is meat. Three popular types of meat are beef, pork, and chicken. Beef comes from cows. Pork comes from pigs. Chicken comes from chickens, but you probably knew that. These animals are raised to become plump and healthy, then they are killed, sometimes at slaughter houses. The meat is then removed from their bodies, cleaned, and made available at a variety of stores and restaurants. Sometimes this process can seem gross, but it is part of an advanced ecological food chain on earth. Just like birds eat worms and tigers eat deer, human beings eat cows and pigs. The main difference is that we don’t eat animals raw. We cook the meat to remove blood, fat, and germs from it. We also season our meat with salt or different kinds of sauces. The end result is food that is very tasty and is healthy for us.

Farmers do not like trespassers. If a farmer sees one, he will sometimes shoot at them with a shotgun that he owns. Trespassing is against the law. Laws are created by government to prevent people from living in fear. They are meant to provide safety for citizens. Our government in America consists of a legislative branch, an executive branch, and a judicial branch. The legislative branch makes laws based on the concerns of citizens they represent. The executive branch consists of the President. This person enforces the law, and he has certain other duties like declaring war and approving bills prepared by members of the legislative branch. The President is also considered the leader of our country. The judicial branch interprets the laws. This branch consists of the courts and the trials held in them. Here a judge and jury determine from evidence presented by lawyers whether someone is guilty of breaking a law. Initial law enforcement takes place among police officers. They are the first people to encounter situations where a law is being broken. If a criminal (law-breaker) becomes too violent or hostile, they will use guns or mace or nightsticks to administer immediate punishment. Their goal is to bring the criminal under control, so that he can receive a punishment determined by members of the judicial branch of government. Punishments mostly include time in jail, but they can also include fines and, in extreme cases, the death penalty. There is controversy surrounding the death penalty.

Children play with toys. This is common to almost all kids. Toys come in a very wide variety. Boys tend to like cars, action figures, and toy weapons. Girls tend to like dolls, toy kitchens, and make-up. Both of them like building or assembling things, be it with Legos, blocks, Play-Doh, or something similar. Toys can be found at most stores, and these days entire stores are dedicated to selling only toys. The most popular of these is Toys ‘R’ Us (with a backwards “R”). Their mascot is Geoffrey the Giraffe. Children love to go to Toys ‘R’ Us and look at the wide variety of toys available. Most children receive the greatest quanitity of toys on their birthdays, or during the holiday season in December. For the majority of children, this holiday is Christmas. For Jewish children, the holiday is Channakuh. Either way, the kid gets presents during this time, and most of these presents are toys.

Christmas is a holiday which has gradually become centered around the character “Santa Claus” and his elves and reindeer. Children are told that Santa’s elves build their toys, and Santa delivers them personally to each house in the world by riding in an airborne sleigh hauled by nine reindeer, including Rudolph the red-nosed reindeer, who leads the way. Another popular Christmas character is Frosty the Snowman. Frosty is basically any snowman that comes to life. So during Christmas, many children build snowmen, and some of them hope that theirs might come to life. But all of these characters are myths. The true origin of Christmas is a celebration of the birth of Jesus, who founded the religion of Christianity a couple of thousand years ago. Many popular Christmas carols deal with his story, such as “Joy to the World” and “Silent Night.”

Other holidays include Thanksgiving, Halloween, and Independence Day. Thanksgiving has become a tradition of preparing large quantities of food for a large gathering of people, mainly family and friends. This meal usually features turkey or ham as the main course. Turkey and ham are both kinds of meat (which I mentioned earlier). The meal usually also consists of dressing and a wide assortment of vegetables (which I also mentioned earlier). The origin of Thanksgiving is usually traced to the days of the pilgrims, who were the first settlers in America. They made peace with the native people, the Indians, and together enjoyed a large feast, thanking God for providing them with such an abundance. (Their concepts of God were probably very different.)

Halloween is the holiday when people dress in costumes to look like other characters. Most of these are children, who go from door to door in different neighborhoods to request candy from the people living there. They usually say “trick or treat” then receive a treat. Very rarely does the person in the house respond with a trick. Halloween has some sort of demonic origin that I am not quite sure about, but the name derives from “All Hallow’s Eve.” I will not say much about Independence Day, but it is the day Americans celebrate the anniversary of our independence from Britain. Most families purchase fireworks during this holiday and set them off in their lawns (which I mentioned earlier).

America gained independence from Britain in the late 1700’s after the Revolutionary War. Britain was hoping to extend its empire across the Atlantic Ocean, but the colonists who settled the territory did not want to be under Britain’s control, with their various taxes and regulations. Both sides were very passionate about their position on the issue, so a war occurred. This war featured a few heroes, including George Washington and Paul Revere. George Washington became America’s first president when we gained independence. I am not sure what happened to Paul Revere. The Declaration of Independence was written before the war by Thomas Jefferson in 1776 and made clear the position of the colonists. It was signed by many important people, including Ben Franklin and John Hancock. Ben Franklin is well-known for many things. One of these is inventing electrical conductors in the form of lightning rods. A famous tale is that he flew a kite with a small piece of metal somewhere on the string during a lightning storm. This was an effective way to test his theory. Another thing Ben Franklin is known for is publishing Poor Richards Almanac. This was like a magazine and contained some of his famous writings and quotations. One famous quote was “Tell me, I forget. Teach me, I remember. Involve me, I learn.” Maybe this had something to do with why he flew that kite.

Trees are one of our most important natural resources. They are made of wood, and wood can be made into a variety of products. Some of the more obvious kinds are furniture, houses, and toothpicks. However, wood can also be made into paper. When I first heard this, I was skeptical, but it is true. Paper is a very important product in our society. Writers and artists have greatly benefited from the invention of paper. With only some paper and a pen or pencil, a writer can produce stories and poems that can captivate readers. They can also write down historical facts about their society. Actually, these writings don’t become historical until years later. At the time, the writings could probably be considered news. Artists use paper for their drawings and paintings. They can also use canvas. Drawings and paintings can be very beautiful. They can depict a wide variety of subjects, including flowers, animals, landscapes, and people. They can be realistic or impressionistic. Some paintings also attempt to convey emotions merely by the way the colors are combined and the brushstrokes are applied. This is a modern or contemporary approach to art. Many people think this approach does not require as much talent as the realistic styles.

I will end my writing here. I have tried to make it very boring, and I hope I have succeeded. There are plenty of boring documents available for you to read. Check your public library for more information. You can also find boring materials at a bookstore or on websites. Sometimes this information can be found in magazines (which I mentioned earlier)."""

For the sake of simplicity, we remove all special characters (except a blank space) and even convert all text into lowercase text. To remove all special characters, we use a special type of language in python, called *Regex*, which stands for *Regular Expressions*. Its like telling python to do tasks which are otherwise hard to execute by code. For example, "take all alphabetical letters, except the letter 'a','b' and 'z', but only uptil you encounter an underscore, after which, you only omit the letter 'z' ", and so on. You get the idea!
Its extremely useful to know regex in Python, and there are great tutorials online. But many times, you can find prewritten regex for your particular task on forums, and so not knowing regex is not necessarily a barrier!

So regex can be used after importing the `re` library.

In [ ]:
import re

We define a function that can remove all special characters from a text, and convert everything to lowercase. It then returns a list containing the words in the text in the exact same sequence as they occur in the text.

In [ ]:
def simplify(text: str): return re.sub(r"\W+|_", " ", text).lower().split() #returns a list of words

In [ ]:
text =simplify(input)
text[:10],len(text)

(['i', 'am', 'writing', 'something', 'yes', 'i', 'plan', 'to', 'make', 'it'],
 2400)

So now, in order to create a model, we will create a data structure, which contains the next occurences of all words, because that's what we're concerned with. So we use a python dictionary, with keys as words, and their corresponding values as a list of all the successive occurences in all of the data (the text above).

The idea behind this is as follows. We will start with a word. Then we look up the successive occurences to the word from this dictionary, and choose a random word. Now this word is our present prediction. Now we will predict the next word through the successive occurences of this present prediction, and this cycle goes on. You can visualize this using the representation below. Each prediction acts as an input for the next prediction!

<figure><center>
<img src='https://drive.google.com/uc?id=1SNp15ppS1BVfoFSU48ox90xtHlG2EzvJ' width='30%'>
</center></figure>


(Credits: Joshua Payne for [this](https://joshua-payne.medium.com/an-introduction-to-recurrent-neural-networks-8151823daeb7) article)

Each word's corresponding occurence list can have one word appear more than once. So we will simply make a list of all the successive occurences in the text - and a word can occur in this list as many times as the combination of the present word and a particular successor occur together. This will infact help us. Since we will choose a random word as our prediction, more the occurences of a particular word, more likely it is to be chosen, right?! So that aligns with our concepts very well.

In [ ]:
def get_occurences(text: list):
    """
    This function takes in a sequence of text as a list, and creates a dictionary of all words in the text, with their values being a list of all words that
    have been seen to occur after the word, in the text.
    Sometimes, a word may not have a successor at all (like the ending word of the text). We don't want our predictions to stop. So we simply add all the words 
    from the text to the value of that word key, so that the model will randomly choose any word from the entire corpus, and start predicting all over again.

    """
    dictionary={k:[] for k in set(text)}
    for k,i in enumerate(text[:-1]):
        dictionary.get(i).append(text[k+1])
    for key in dictionary: 
        if not dictionary.get(key): dictionary.get(key).extend(list(set(text)))
    
    return dictionary

In [ ]:
text_dict=get_occurences(text)
print(text_dict)

{'controversy': ['surrounding'], 'convey': ['emotions'], 'why': ['he'], 'make': ['it', 'and', 'clothing', 'up', 'it'], 'stewed': ['and'], 'main': ['ingredient', 'difference', 'course'], 'person': ['enforces', 'in'], 'native': ['people'], 'always': ['consist'], 'hoping': ['to'], 'various': ['taxes'], 'situations': ['where'], 'deer': ['human'], 'lightning': ['rods', 'storm'], 'sports': ['card'], 'realistic': ['or', 'styles'], 'many': ['cars', 'types', 'families', 'of', 'parents', 'people', 'years', 'medical', 'children', 'popular', 'important', 'things', 'people'], 'mentioned': ['earlier', 'earlier', 'earlier', 'earlier', 'earlier', 'earlier', 'earlier'], 'sometimes': ['notice', 'even', 'see', 'at', 'this', 'shoot', 'this'], 'helps': ['give'], 'killed': ['sometimes'], 'bodies': ['cleaned'], 'jail': ['but'], 'different': ['kinds', 'kinds', 'types', 'kinds', 'halloween', 'neighborhoods'], 'give': ['them'], 'cover': ['a'], 'surrounding': ['the'], 'giraffe': ['children'], 'later': ['at'], 'w

We can now finally build our Markov Model. We will randomly choose a word, and continuously predict the following words. To choose random items, we will use a library called `random`.

In [ ]:
import random

In [ ]:
word=random.choice(text) #initialize with a random word
word

'to'

In [ ]:
for _ in range(1000): 
    print(word,end=' ')
    word=random.choice(text_dict[word])

to toys r their mascot is geoffrey the death penalty children this holiday is the government punishments mostly include such an optional item for us and poems that is made peace with the laws are very large patch of subjects including george washington became america s after the declaration of subjects including george washington became america gained independence day but the position of children the kid gets presents during the law laws this had something similar toys on the birth of a large amount of things as either fruits or hostile they almost always consist of children who founded the colors are items for more efficiently through several brands including marlboro salem and add butter to prepare potatoes in less time and he flew that we gained independence was written i sometimes shoot at a teenager in water to wash clothes into a law laws are used in the holiday season our country the most people use a very boring and lettuce for providing them off in different kinds of toys the 

And, with that, you have successfully built your text generation system.

You would notice that this generated text does not make a lot of sense (Though it does make sense somewhere, atleast in short phrases!). Thats because the model does not understand contexts. It only predicts words based on the current word, and does not consider the past words at all! This is called a *memoryless model*, meaning it does not have memory of past words, and only works based on the current input. 

It also makes random predictions based on probabilities. This randomness often creates meaningless predictions. Nowadays, we use much more sophisticated models, such as Recurrent Neural Networks, that can understand very complex contexts and even hidden meanings in the text. 

## Review
In this session, we learnt about two very interesting Bayesian models, or probabilistic models - the Naive Bayes Classifier and the Markov Chain Model. These both work in different ways and have great applications in their own domains. We built a classifier using no iterative learning, and only through analyzing the frequencies of occurence of a particular value.

Simple probabilistic models are very crude, because of the fact that there exists unguided randomness in these models. But yet, there does exist logical sense in these models.

## Review Questions:
These are non-evaluative, but highly recommended to go through. Make sure you clearly know the answer to each of these concepts. The answers to all these questions are somewhere in this notebook, so if you find yourself unclear with a concept, go back up and find the answer!

1. What is the significance of *probability* in Machine Learning?
2. Explain what $P(pred)$,$ P(data)$ and $P(data|pred)$ mean in essence?, and how are they related to our classification model?
3. How have we represented each of $P(pred)$,$ P(data)$ and $P(data|pred)$ in code? Meaning, what data structures have been used, and how are they organised?
4. Go to the section where we define `get_prediction_accuracy` function. Understand the working of this code, and in 4-5 lines of plain English, explain the working in pseudo terms. 
5. Can you give 5 different everyday examples, where the Markov Chain can be used?
6. What is Regex?
7. When would a Markov Chain result in an infinite looping situation?


# Exercise (Evaluative):


## Problem 1: Extended Naive Bayes Classifier

In the above problem, we simply dropped all continuous variables (Why?). But there are ways to convert continuous variables into categorical variables. One such method is *binning*, which means, we divide the data into separate categories, or bins, based on their value. For example, values from 0 to 5 lie in one bin, 5 to 10 in another and so on. So let us try to include continuous variables as well into our data, and see if we can get a better Bayesian accuracy.

So first do it for the variable Age. According to the dataset Age lies between 0 and 85. So you can create bins of 10 years (ie, 0 to 10, 10 to 20 and so on).

Hint: You can do it easily using list comprehensions:

```
train['Age'] = [some_function(i) for i in train['Age'] #Do it for the testing dataset too
```

In [1]:
%cd 
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json 

!kaggle datasets download -d teejmahal20/airline-passenger-satisfaction
!unzip airline-passenger-satisfaction.zip 
!mkdir airlines && mv test.csv airlines && mv train.csv airlines
%cd airlines
!ls

/root


Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 72 bytes
airline-passenger-satisfaction.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  airline-passenger-satisfaction.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               
mkdir: cannot create directory ‘airlines’: File exists
/root/airlines
shakespeare-text.zip  test.csv	text.txt  train.csv


In [2]:
import pandas as pd

##**Age Bins** 
Test accuracy = 72.56%

In [3]:
train=pd.read_csv('train.csv',index_col=0).drop(['id','Departure Delay in Minutes','Arrival Delay in Minutes','Flight Distance'],axis=1) 
test=pd.read_csv('test.csv',index_col=0).drop(['id','Departure Delay in Minutes','Arrival Delay in Minutes','Flight Distance'],axis=1)

In [4]:
train.head()

,Gender,Customer Type,Age,Type of Travel,Class,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,3,4,3,1,5,3,5,5,4,3,4,4,5,5,neutral or dissatisfied
1,Male,disloyal Customer,25,Business travel,Business,3,2,3,3,1,3,1,1,1,5,3,1,4,1,neutral or dissatisfied
2,Female,Loyal Customer,26,Business travel,Business,2,2,2,2,5,5,5,5,4,3,4,4,4,5,satisfied
3,Female,Loyal Customer,25,Business travel,Business,2,5,5,5,2,2,2,2,2,5,3,1,4,2,neutral or dissatisfied
4,Male,Loyal Customer,61,Business travel,Business,3,3,3,3,4,5,5,3,3,4,4,3,3,3,satisfied


In [5]:
def age_bin(age_conti):
  return (int)(age_conti/10)

In [6]:
train['Age'] = [age_bin(i) for i in train['Age']]
test['Age'] = [age_bin(i) for i in test['Age']]

In [7]:
def categorify(df):
    df.dictionary={}
    for col in df: 
        numerical_categories = {k:i for i,k in enumerate(sorted(df[col].unique()))}
        df[col]=[numerical_categories[i] for i in df[col]]
        df.dictionary[col]=numerical_categories

categorify(train)
categorify(test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [8]:
train.head()

,Gender,Customer Type,Age,Type of Travel,Class,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,satisfaction
0,1,0,1,1,2,3,4,3,1,5,3,5,5,4,3,3,4,5,5,0
1,1,1,2,0,0,3,2,3,3,1,3,1,1,1,5,2,1,4,1,0
2,0,0,2,0,0,2,2,2,2,5,5,5,5,4,3,3,4,4,5,1
3,0,0,2,0,0,2,5,5,5,2,2,2,2,2,5,2,1,4,2,0
4,1,0,6,0,0,3,3,3,3,4,5,5,3,3,4,3,3,3,3,1


In [9]:
train.dictionary

{'Age': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8},
 'Baggage handling': {1: 0, 2: 1, 3: 2, 4: 3, 5: 4},
 'Checkin service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Class': {'Business': 0, 'Eco': 1, 'Eco Plus': 2},
 'Cleanliness': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Customer Type': {'Loyal Customer': 0, 'disloyal Customer': 1},
 'Departure/Arrival time convenient': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Ease of Online booking': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Food and drink': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Gate location': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Gender': {'Female': 0, 'Male': 1},
 'Inflight entertainment': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Inflight service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Inflight wifi service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Leg room service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'On-board service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Online boarding': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},

In [10]:
def get_ppred(df,tgt='satisfaction'):
    f"get P(pred) as a dictionary, where tgt is the target variable and does not count as a feature." #this is a comment for the user
    p_pred=df[tgt].value_counts().to_dict()
    for p in p_pred: p_pred[p]/=len(df) # convert the frequencies into a ratio of occurence
    return p_pred

In [11]:
p_pred = get_ppred(train,'satisfaction')
# p_pred

In [12]:
def get_pdata(df,tgt='satisfaction'): return {k:get_ppred(df,tgt=k) for k in df.keys().drop(tgt)}

In [13]:
p_data=get_pdata(train)
# p_data

In [14]:
p_data['Customer Type']

{0: 0.8173217585463505, 1: 0.18267824145364953}

In [15]:
sum(p_data['Customer Type'].values())

1.0

In [16]:
for key in p_data: 
    assert abs(sum(p_data[key].values())-1)<1e-6, f"probabilities of the key: {key} does not sum up to 1"

In [17]:
def get_pdata_given_pred(df,tgt='satisfaction'): return {k:calc_pdata_given_pred(df,tgt,key=k) for k in df[tgt].unique()}

def calc_pdata_given_pred(df,tgt,key):
    ret={f:None for f in df if f!=tgt}
    l=len(df)
    df=df[df[tgt]==key]
    for f in ret:
        conditional_probs=df[f].value_counts().to_dict()
        for o in conditional_probs: conditional_probs[o]/=l
        ret[f]=conditional_probs
    return ret

In [18]:
p_data_given_pred = get_pdata_given_pred(train)
# p_data_given_pred

In [19]:
for feature in train.keys().drop('satisfaction'): 
    assert p_data_given_pred[1][feature].get(0,0) + p_data_given_pred[0][feature].get(0,0) == p_data[feature].get(0)

In [20]:
def get_prediction_accuracy(df,tgt,p_data,p_pred,p_data_given_pred):
    probability_of_being_1=[]
    probability_of_being_0=[]
    features=df.keys().drop(tgt)
    for _,row in df.iterrows(): 
        probability_of_being_1.append(calculate_prob(row,features,p_data,p_pred,p_data_given_pred,tgt=1))
        probability_of_being_0.append(calculate_prob(row,features,p_data,p_pred,p_data_given_pred,tgt=0))
    
    pred=[p_one>=p_zero for p_one,p_zero in zip(probability_of_being_1,probability_of_being_0)]
    return (pred==df[tgt]).mean()

def calculate_prob(row: pd.Series,features,p_data,p_pred,p_data_given_pred,tgt=1):
    p_pred_tgt = p_pred[tgt] #p_pred for target=1
    p_data_1=1. #initializing value
    p_data_given_pred_1 =1. #initialize

    for f in features:
        p_data_1*=p_data[f][row[f]]
        p_data_given_pred_1*=p_data_given_pred[tgt][f].get(row[f],0)

    return p_pred_tgt*p_data_given_pred_1/p_data_1

In [21]:
get_prediction_accuracy(train,'satisfaction',p_data,p_pred,p_data_given_pred)

0.8210752232830305

In [22]:
get_prediction_accuracy(test,'satisfaction',p_data,p_pred,p_data_given_pred)

0.725669849091469

Build a Naive Bayes classifier with the new dataframes now. Is there a significant improvement/deterioration in the accuracy? (There need not necessarily be, thats okay!)

Next, build a classifier with various combinations of continuous variables, in addition to all categorical variables. Bin the continuous variables using an appropriate bin size. Keep the following things in mind - the number of bins should not be too large, or there is no difference between this and a continuous variable (both take large number of values, hence the corresponding probabilities of occurence would be very low), or too small either (otherwise there is loss of information). A moderate number of bins is sufficient. Try to push the accuracy as far as possible, and report which combination of features gives the best informatoin about customer satisfaction. 

As a final step, try to think of which *categorical* variables do not affect customer satisfaction (through your own personal understanding), and remove upto 3 such variables, and see how far the accuracy can be achieved. This answer is subjective, so accuracies may vary for everyone.

For your reference, the following are all the features of the Airplane Passenger Satisfaction Dataset.

```
['id', 'Gender', 'Customer Type', 'Age', 'Type of Travel', 'Class',
'Flight Distance', 'Inflight wifi service',
'Departure/Arrival time convenient', 'Ease of Online booking',
'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
'Inflight entertainment', 'On-board service', 'Leg room service',
'Baggage handling', 'Checkin service', 'Inflight service',
'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
'satisfaction']
```
`id` will be always dropped (removed) from the dataframe

## **All categorical features + continuous variables bins**
Best results (testing accuracy = 72.04%) are obtained by taking all features except 'id','Age','Arrival Delay in Minutes'

In [23]:
train=pd.read_csv('train.csv',index_col=0).drop(['id','Age','Arrival Delay in Minutes'],axis=1) 
test=pd.read_csv('test.csv',index_col=0).drop(['id','Age','Arrival Delay in Minutes'],axis=1) 

In [24]:
train.head()

,Gender,Customer Type,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,satisfaction
0,Male,Loyal Customer,Personal Travel,Eco Plus,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,neutral or dissatisfied
1,Male,disloyal Customer,Business travel,Business,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,neutral or dissatisfied
2,Female,Loyal Customer,Business travel,Business,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,satisfied
3,Female,Loyal Customer,Business travel,Business,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,neutral or dissatisfied
4,Male,Loyal Customer,Business travel,Business,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,satisfied


In [25]:
max(train['Departure Delay in Minutes']), min(train['Departure Delay in Minutes']), max(test['Departure Delay in Minutes']), min(test['Departure Delay in Minutes'])

(1592, 0, 1128, 0)

In [26]:
# max(train['Arrival Delay in Minutes']), min(train['Arrival Delay in Minutes']), max(test['Arrival Delay in Minutes']), min(test['Arrival Delay in Minutes'])

In [27]:
max(train['Flight Distance']), min(train['Flight Distance']), max(test['Flight Distance']), min(test['Flight Distance'])

(4983, 31, 4983, 31)

In [28]:
def age_bin(age_conti):
  return (int)(age_conti/10.0)

def time_bin(time_conti):
  return (int)(time_conti/200.0)

def dist_bin(dist_conti):
  return (int)(dist_conti/500.0)

In [29]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)
# train['Age'] = [age_bin(i) for i in train['Age']]
# test['Age'] = [age_bin(i) for i in test['Age']]
train['Departure Delay in Minutes'] = [time_bin(i) for i in train['Departure Delay in Minutes']]
test['Departure Delay in Minutes'] = [time_bin(i) for i in test['Departure Delay in Minutes']]
# train['Arrival Delay in Minutes'] = [time_bin(i) for i in train['Arrival Delay in Minutes']]
# test['Arrival Delay in Minutes'] = [time_bin(i) for i in test['Arrival Delay in Minutes']]
train['Flight Distance'] = [dist_bin(i) for i in train['Flight Distance']]
test['Flight Distance'] = [dist_bin(i) for i in test['Flight Distance']]

In [30]:
# train

In [31]:
categorify(train)
categorify(test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [32]:
train.head()

,Gender,Customer Type,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,satisfaction
0,1,0,1,2,0,3,4,3,1,5,3,5,5,4,3,3,4,5,5,0,0
1,1,1,0,0,0,3,2,3,3,1,3,1,1,1,5,2,1,4,1,0,0
2,0,0,0,0,2,2,2,2,2,5,5,5,5,4,3,3,4,4,5,0,1
3,0,0,0,0,1,2,5,5,5,2,2,2,2,2,5,2,1,4,2,0,0
4,1,0,0,0,0,3,3,3,3,4,5,5,3,3,4,3,3,3,3,0,1


In [33]:
train.dictionary

{'Baggage handling': {1: 0, 2: 1, 3: 2, 4: 3, 5: 4},
 'Checkin service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Class': {'Business': 0, 'Eco': 1, 'Eco Plus': 2},
 'Cleanliness': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Customer Type': {'Loyal Customer': 0, 'disloyal Customer': 1},
 'Departure Delay in Minutes': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7},
 'Departure/Arrival time convenient': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Ease of Online booking': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Flight Distance': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9},
 'Food and drink': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Gate location': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Gender': {'Female': 0, 'Male': 1},
 'Inflight entertainment': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Inflight service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Inflight wifi service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Leg room service': {0: 0, 1: 1, 2: 2, 3: 3

In [34]:
p_pred = get_ppred(train,'satisfaction')
# p_pred

In [35]:
p_data=get_pdata(train)
# p_data

In [36]:
p_data['Customer Type']

{0: 0.8173217585463505, 1: 0.18267824145364953}

In [37]:
sum(p_data['Customer Type'].values())

1.0

In [38]:
for key in p_data: 
    assert abs(sum(p_data[key].values())-1)<1e-6, f"probabilities of the key: {key} does not sum up to 1"

In [39]:
p_data_given_pred = get_pdata_given_pred(train)
# p_data_given_pred

In [40]:
for feature in train.keys().drop('satisfaction'): 
    assert abs(p_data_given_pred[1][feature].get(0,0) + p_data_given_pred[0][feature].get(0,0) - p_data[feature].get(0))<1e-6

In [41]:
get_prediction_accuracy(train,'satisfaction',p_data,p_pred,p_data_given_pred)

0.8153199106867878

In [42]:
get_prediction_accuracy(test,'satisfaction',p_data,p_pred,p_data_given_pred)

0.7204342469972282

## **All categorical features + All continuous variables bins**

In [43]:
train=pd.read_csv('train.csv',index_col=0).drop(['id'],axis=1) 
test=pd.read_csv('test.csv',index_col=0).drop(['id'],axis=1) 

In [44]:
train.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0,satisfied
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0,satisfied


In [45]:
max(train['Departure Delay in Minutes']), min(train['Departure Delay in Minutes']), max(test['Departure Delay in Minutes']), min(test['Departure Delay in Minutes'])

(1592, 0, 1128, 0)

In [46]:
max(train['Arrival Delay in Minutes']), min(train['Arrival Delay in Minutes']), max(test['Arrival Delay in Minutes']), min(test['Arrival Delay in Minutes'])

(1584.0, 0.0, 1115.0, 0.0)

In [47]:
max(train['Flight Distance']), min(train['Flight Distance']), max(test['Flight Distance']), min(test['Flight Distance'])

(4983, 31, 4983, 31)

In [48]:
def age_bin(age_conti):
  return (int)(age_conti/10.0)

def time_bin(time_conti):
  return (int)(time_conti/200.0)

def dist_bin(dist_conti):
  return (int)(dist_conti/500.0)

In [49]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)
train['Age'] = [age_bin(i) for i in train['Age']]
test['Age'] = [age_bin(i) for i in test['Age']]
train['Departure Delay in Minutes'] = [time_bin(i) for i in train['Departure Delay in Minutes']]
test['Departure Delay in Minutes'] = [time_bin(i) for i in test['Departure Delay in Minutes']]
train['Arrival Delay in Minutes'] = [time_bin(i) for i in train['Arrival Delay in Minutes']]
test['Arrival Delay in Minutes'] = [time_bin(i) for i in test['Arrival Delay in Minutes']]
train['Flight Distance'] = [dist_bin(i) for i in train['Flight Distance']]
test['Flight Distance'] = [dist_bin(i) for i in test['Flight Distance']]

In [50]:
# train

In [51]:
categorify(train)
categorify(test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [52]:
train.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,1,0,1,1,2,0,3,4,3,1,5,3,5,5,4,3,3,4,5,5,0,0,0
1,1,1,2,0,0,0,3,2,3,3,1,3,1,1,1,5,2,1,4,1,0,0,0
2,0,0,2,0,0,2,2,2,2,2,5,5,5,5,4,3,3,4,4,5,0,0,1
3,0,0,2,0,0,1,2,5,5,5,2,2,2,2,2,5,2,1,4,2,0,0,0
4,1,0,6,0,0,0,3,3,3,3,4,5,5,3,3,4,3,3,3,3,0,0,1


In [53]:
train.dictionary

{'Age': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8},
 'Arrival Delay in Minutes': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7},
 'Baggage handling': {1: 0, 2: 1, 3: 2, 4: 3, 5: 4},
 'Checkin service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Class': {'Business': 0, 'Eco': 1, 'Eco Plus': 2},
 'Cleanliness': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Customer Type': {'Loyal Customer': 0, 'disloyal Customer': 1},
 'Departure Delay in Minutes': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7},
 'Departure/Arrival time convenient': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Ease of Online booking': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Flight Distance': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9},
 'Food and drink': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Gate location': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Gender': {'Female': 0, 'Male': 1},
 'Inflight entertainment': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Inflight service': {0:

In [54]:
p_pred = get_ppred(train,'satisfaction')
# p_pred

In [55]:
p_data=get_pdata(train)
# p_data

In [56]:
p_data['Customer Type']

{0: 0.8173217585463505, 1: 0.18267824145364953}

In [57]:
sum(p_data['Customer Type'].values())

1.0

In [58]:
for key in p_data: 
    assert abs(sum(p_data[key].values())-1)<1e-6, f"probabilities of the key: {key} does not sum up to 1"

In [59]:
p_data_given_pred = get_pdata_given_pred(train)
# p_data_given_pred

In [60]:
for feature in train.keys().drop('satisfaction'): 
    assert abs(p_data_given_pred[1][feature].get(0,0) + p_data_given_pred[0][feature].get(0,0) - p_data[feature].get(0))<1e-6

In [61]:
get_prediction_accuracy(train,'satisfaction',p_data,p_pred,p_data_given_pred)

0.8023367724052972

In [62]:
get_prediction_accuracy(test,'satisfaction',p_data,p_pred,p_data_given_pred)

0.709809054511857

## **Removing upto 3 categorical variables**
Best results (testing accuracy = 76.14%) are obtained by taking all features except 'id', 'Gender','Type of Travel', 'Gate location', 'Age', 'Arrival Delay in Minutes'

In [63]:
train=pd.read_csv('train.csv',index_col=0).drop(['id', 'Gender','Type of Travel', 'Gate location', 'Age','Arrival Delay in Minutes'],axis=1) 
test=pd.read_csv('test.csv',index_col=0).drop(['id', 'Gender','Type of Travel', 'Gate location', 'Age','Arrival Delay in Minutes'],axis=1) 

In [64]:
train.head()

,Customer Type,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,satisfaction
0,Loyal Customer,Eco Plus,460,3,4,3,5,3,5,5,4,3,4,4,5,5,25,neutral or dissatisfied
1,disloyal Customer,Business,235,3,2,3,1,3,1,1,1,5,3,1,4,1,1,neutral or dissatisfied
2,Loyal Customer,Business,1142,2,2,2,5,5,5,5,4,3,4,4,4,5,0,satisfied
3,Loyal Customer,Business,562,2,5,5,2,2,2,2,2,5,3,1,4,2,11,neutral or dissatisfied
4,Loyal Customer,Business,214,3,3,3,4,5,5,3,3,4,4,3,3,3,0,satisfied


In [65]:
max(train['Departure Delay in Minutes']), min(train['Departure Delay in Minutes']), max(test['Departure Delay in Minutes']), min(test['Departure Delay in Minutes'])

(1592, 0, 1128, 0)

In [66]:
# max(train['Arrival Delay in Minutes']), min(train['Arrival Delay in Minutes']), max(test['Arrival Delay in Minutes']), min(test['Arrival Delay in Minutes'])

In [67]:
max(train['Flight Distance']), min(train['Flight Distance']), max(test['Flight Distance']), min(test['Flight Distance'])

(4983, 31, 4983, 31)

In [68]:
def age_bin(age_conti):
  return (int)(age_conti/10.0)

def time_bin(time_conti):
  return (int)(time_conti/200.0)

def dist_bin(dist_conti):
  return (int)(dist_conti/500.0)

In [69]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)
# train['Age'] = [age_bin(i) for i in train['Age']]
# test['Age'] = [age_bin(i) for i in test['Age']]
train['Departure Delay in Minutes'] = [time_bin(i) for i in train['Departure Delay in Minutes']]
test['Departure Delay in Minutes'] = [time_bin(i) for i in test['Departure Delay in Minutes']]
# train['Arrival Delay in Minutes'] = [time_bin(i) for i in train['Arrival Delay in Minutes']]
# test['Arrival Delay in Minutes'] = [time_bin(i) for i in test['Arrival Delay in Minutes']]
train['Flight Distance'] = [dist_bin(i) for i in train['Flight Distance']]
test['Flight Distance'] = [dist_bin(i) for i in test['Flight Distance']]

In [70]:
# train.head()

In [71]:
categorify(train)
categorify(test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [72]:
train.head()

,Customer Type,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,satisfaction
0,0,2,0,3,4,3,5,3,5,5,4,3,3,4,5,5,0,0
1,1,0,0,3,2,3,1,3,1,1,1,5,2,1,4,1,0,0
2,0,0,2,2,2,2,5,5,5,5,4,3,3,4,4,5,0,1
3,0,0,1,2,5,5,2,2,2,2,2,5,2,1,4,2,0,0
4,0,0,0,3,3,3,4,5,5,3,3,4,3,3,3,3,0,1


In [73]:
train.dictionary

{'Baggage handling': {1: 0, 2: 1, 3: 2, 4: 3, 5: 4},
 'Checkin service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Class': {'Business': 0, 'Eco': 1, 'Eco Plus': 2},
 'Cleanliness': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Customer Type': {'Loyal Customer': 0, 'disloyal Customer': 1},
 'Departure Delay in Minutes': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7},
 'Departure/Arrival time convenient': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Ease of Online booking': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Flight Distance': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9},
 'Food and drink': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Inflight entertainment': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Inflight service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Inflight wifi service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Leg room service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'On-board service': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5},
 'Online boarding':

In [74]:
p_pred = get_ppred(train,'satisfaction')
# p_pred

In [75]:
p_data=get_pdata(train)
# p_data

In [76]:
# p_data['Customer Type']

In [77]:
# sum(p_data['Customer Type'].values())

In [78]:
for key in p_data: 
    assert abs(sum(p_data[key].values())-1)<1e-6, f"probabilities of the key: {key} does not sum up to 1"

In [79]:
p_data_given_pred = get_pdata_given_pred(train)
# p_data_given_pred

In [80]:
for feature in train.keys().drop('satisfaction'): 
    assert abs(p_data_given_pred[1][feature].get(0,0) + p_data_given_pred[0][feature].get(0,0) - p_data[feature].get(0))<1e-6

In [81]:
get_prediction_accuracy(train,'satisfaction',p_data,p_pred,p_data_given_pred)

0.823972128118263

In [82]:
get_prediction_accuracy(test,'satisfaction',p_data,p_pred,p_data_given_pred)

0.761356636895596

## Problem 2: Markov? Shakespeare? What is in the name!

We learnt that the next prediction of a Markov Model only depends on the present prediction/value, i.e. the model is a *memoryless* model.

Let us build a memory model, ie a model, which also takes into account the predecessor of the current prediction, along with the current prediction itself. Meaning, what would be the next word, given the last *2* predictions?!

But this most probably won't work on the text we wrote above (the boring text), because it contains only 2400 words. So there is a very low probability that the successor of two consecutive words will have more than one possible options. We will end up getting the exact same text as the text we wrote, or almost on the same lines. 

Let us download a bigger text file - how about a whole Shakespeare play. And we will try to build a model that can predict text in the style of Shakespeare!

Let us download this dataset and put it in a string.

In [83]:
!kaggle datasets download -d adarshpathak/shakespeare-text
!unzip shakespeare-text

shakespeare-text.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  shakespeare-text.zip
replace text.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: text.txt                


In [84]:
from pathlib import Path
txt = Path('text.txt').read_text()

Now we need to define our functions. We've written the code to incorporate two consecutive words at once. 

In [85]:
def simplify(text: str):
    words = re.sub(r"\W+|_", " ", text).lower().split()
    return [' '.join((words[i],words[i+1])) for i,_ in enumerate(words[:-1])], words 

In [86]:
def get_occurences(text: list,original_wordlist):

    dictionary={k:[] for k in set(text)}
    # dictionary['unk']=text
    for k,i in enumerate(text[:-1]):
        dictionary.get(i).append(text[k+1])
    for key in dictionary: 
        if not dictionary.get(key): dictionary.get(key).extend(text)
    
    return dictionary

Continue the code from here, and write the loop function to generate the Markov Chain Model's predictions. 

Randomly choose a word, and predict the next 100 words.

In [87]:
import re
import random

In [94]:
double_wordList, single_wordList=simplify(txt)

In [95]:
text_dict=get_occurences(double_wordList, single_wordList)

In [99]:
prediction_words=random.choice(double_wordList)
prediction_words

'rushing on'

In [100]:
ans=prediction_words
for _ in range(100): 
    word=random.choice(text_dict[prediction_words]).split()[-1]
    ans=ans+" "+word
    prediction_words=ans.split()[-2]+" "+ans.split()[-1]

In [101]:
ans

'rushing on this man as you have power to take from you marcius will the country for these rites of knighthood else will i read no other way unless thou yield the ghost but still the nurse cursed in the which thou saw st sink sit down for now thou art a coward as you do not sue to live john of gaunt sister farewell i will follow you derby we have found scaling his present bearing with his oath cleomenes you tempt him over much paulina so be thy wife elbow ay sir by the selfsame hand that made gape the'